In [0]:
import os

# Required for Spark ML + MLflow on Databricks Serverless
os.environ["MLFLOW_DFS_TMP"] = "/Volumes/ai_trust_catalog/churn_trust/mlflow_volume"

In [0]:
# Output of Model-1 feature pipeline
gold_df = spark.table(
    "ai_trust_catalog.churn_trust.gold_churn_ml"
)

In [0]:
val_df = gold_df.filter("dataset_split = 'validation'")

In [0]:
import mlflow
import mlflow.spark

# Load Model-1 directly using known successful run_id
MODEL_1_RUN_ID = "6fd9db5ae03d445a81f40ccce6c94335"

model_1 = mlflow.spark.load_model(
    f"runs:/{MODEL_1_RUN_ID}/model_1"
)

print("Loaded Model-1 from run:", MODEL_1_RUN_ID)

In [0]:
# Generate predictions + probabilities
pred_df = model_1.transform(val_df)

In [0]:
from pyspark.sql.functions import col, when, udf
from pyspark.sql.types import DoubleType

# Trust label: 1 = Model-1 correct, 0 = wrong
trust_df = pred_df.withColumn(
    "trust_label",
    when(col("churn") == col("prediction"), 1).otherwise(0)
)

# Confidence = probability of predicted class
def confidence(prob, pred):
    return float(prob[int(pred)])

confidence_udf = udf(confidence, DoubleType())

trust_df = trust_df.withColumn(
    "prediction_confidence",
    confidence_udf(col("probability"), col("prediction"))
)

# Uncertainty = inverse of confidence
trust_df = trust_df.withColumn(
    "prediction_uncertainty",
    1 - col("prediction_confidence")
)

In [0]:
trust_final_df = trust_df.select(
    "prediction_confidence",
    "prediction_uncertainty",
    "trust_label"
)

In [0]:
%sql
SHOW TABLES IN ai_trust_catalog.churn_trust;

In [0]:
%sql
DESCRIBE TABLE ai_trust_catalog.churn_trust.gold_ai_trust_scores;

In [0]:
%sql
DESCRIBE TABLE ai_trust_catalog.churn_trust.model_1_predictions;

In [0]:
%sql
DESCRIBE TABLE ai_trust_catalog.churn_trust.gold_churn_ml;

In [0]:
%sql
SELECT trust_prediction, COUNT(*) AS cnt
FROM ai_trust_catalog.churn_trust.gold_ai_trust_scores
GROUP BY trust_prediction;

In [0]:
%sql
SELECT
  CASE
    WHEN ai_trust_score >= 0.8 THEN 'high_trust'
    WHEN ai_trust_score >= 0.5 THEN 'medium_trust'
    ELSE 'low_trust'
  END AS trust_bucket,
  COUNT(*) AS cnt
FROM ai_trust_catalog.churn_trust.gold_ai_trust_scores
GROUP BY trust_bucket;

In [0]:
%sql
DESCRIBE TABLE ai_trust_catalog.churn_trust.gold_ai_trust_scores;